In [ ]:
# Instalando as libs

%pip install -q google-genai
%pip install -q google-adk

In [ ]:
# Importações

from google.adk.sessions import InMemorySessionService
from IPython.display import display, Markdown
from google.adk.tools import google_search
from google.adk.runners import Runner
from google.adk.agents import Agent
from google.colab import userdata
import plotly.graph_objects as go
#import matplotlib.pyplot as plt
from google.genai import types
from datetime import date
from google import genai
import textwrap
import requests
import warnings
import json
import os

warnings.filterwarnings("ignore")

In [ ]:
# Configura o cliente da SDK do Gemini

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

client = genai.Client()

MODEL = "gemini-2.0-flash"

In [ ]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [ ]:
# Agente 01 - Buscador de Métodos Financeiros
def agente_buscador(renda_mensal):

    buscador = Agent(
        name="agente_buscador",
        model="gemini-2.0-flash",
        instruction="""
        Você é um assistente de pesquisa especializado em finanças pessoais.
        Sua tarefa é usar a ferramenta de busca do Google (google_search) para encontrar os 5 métodos mais efetivos
         e amplamente recomendados para organização financeira pessoal, considerando a renda mensal abaixo.

        Ao realizar a pesquisa, priorize métodos que sejam:
        - Especificamente adequados para o nível de renda fornecido (por exemplo, métodos para baixa renda, renda média, alta renda).
        - Os métodos devem ser aplicáveis à realidade brasileira (ex.: considerar inflação, juros altos, etc.).
        - Baseados em princípios sólidos de gestão financeira.
        - Frequentemente mencionados e bem avaliados por especialistas e usuários.
        - Que ofereçam exemplos ou estudos de caso relevantes para o nível de renda em questão.
        - Descarte métodos genéricos ou irreais (ex.: "ganhe mais dinheiro" sem ações concretas).

        Retorne uma lista concisa com o nome de cada método encontrado e um breve resumo de sua abordagem principal,
        mencionando como ele se aplica a faixas de renda semelhantes à do usuário, se essa informação estiver disponível.
        A lista deve estar em formato Markdown.
        """,
        description="Agente que busca métodos financeiros indicados a uma renda mensal no Google",
        tools=[google_search]
    )

    return call_agent(buscador, f"Renda mensal: {renda_mensal}")

In [ ]:
# Agente 02 - Matemático
def agente_matematico(renda_mensal, gastos):

    matematico = Agent(
        name="agente_matematico",
        model="gemini-2.0-flash",
        instruction="""
        Você é um analista matemático financeiro pessoal. Sua tarefa é receber a renda mensal do usuário
        e seus gastos detalhados por categoria. Com base nessas informações,
        você deve realizar os seguintes cálculos e análises:

        1. Calcule o total de gastos mensais.
        2. Calcule a porcentagem que cada categoria de gasto representa em relação à renda mensal total.
        3. Calcule a diferença entre a renda mensal e o total de gastos (saldo mensal).
        4. Calcule a taxa de poupança mensal atual (se o saldo for positivo).
        5. Identifique as três maiores categorias de gastos em termos percentuais.

        Retorne um dicionário ou estrutura de dados organizada contendo:
        - Renda Mensal: [valor]
        - Total de Gastos Mensais: [valor]
        - Porcentagem de Gasto por Categoria: {categoria1: [porcentagem], categoria2: [porcentagem], ...}
        - Saldo Mensal: [valor]
        - Taxa de Poupança Mensal: [porcentagem] (se aplicável)
        - Maiores Categorias de Gasto: [categoria1, categoria2, categoria3]
        """,
        description="Agente que analisa as finanças do usuário.",
    )

    return call_agent(matematico, f"Renda mensal: {renda_mensal}\nGastos: {gastos}")

In [ ]:
# Agente 03 - Analista
def agente_analista(dados, metodos):

    analista = Agent(
        name="agente_analista",
        model="gemini-2.0-flash",
        instruction="""
        Você é um consultor financeiro pessoal com expertise em diferentes métodos de organização financeira.
        Sua tarefa é analisar os dados financeiros do usuário e os métodos de organização financeira encontrados.

        Com base nessa análise, você deve:
        1. Avaliar a situação financeira atual do usuário, identificando pontos fortes e áreas de melhoria (por exemplo, alta porcentagem de gastos em lazer, baixa taxa de poupança).
        2. Comparar os padrões de gastos do usuário com os princípios de cada método de organização financeira.
        3. Selecionar o método que melhor se adapta ao perfil financeiro do usuário, considerando seus maiores gastos,
        sua taxa de poupança atual e o potencial de implementação do método. Justifique sua escolha com base nos dados analisados.

        Retorne um dicionário ou estrutura de dados contendo:
        - Análise da Situação Financeira: [um breve resumo da situação]
        - Método Recomendado: [nome do método]
        - Justificativa do Método: [explicação de por que este método é o mais adequado]
        O retorno deve estar em formato Markdown.
        """,
        description="Agente que analisa as finanças do usuário.",
    )

    return call_agent(analista, f"Dados: {dados}\nMétodos: {metodos}")

In [ ]:
# Agente 04 - Orientador
def agente_orientador(dados, analise):

    orientador = Agent(
        name="agente_orientador",
        model="gemini-2.0-flash",
        instruction="""
        Você é um orientador financeiro pessoal dedicado a ajudar os usuários a melhorar sua saúde financeira.
        Sua tarefa é receber a análise do Agente Analista e os dados financeiros do Agente Matemático para fornecer orientações claras e práticas ao usuário,
        juntamente com os dados estruturados necessários para a criação de gráficos.

        Você deve retornar um dicionário contendo as seguintes chaves:

        1.  'orientacao': Uma string contendo:
            A explicação detalhada do método de organização financeira recomendado, seus princípios e como o usuário pode implementá-lo.
            Sugestões específicas e acionáveis nos hábitos de gastos do usuário, com foco nas maiores categorias identificadas.
            Quantifique o potencial de economia dessas mudanças, se possível.
            A mensagem deve ser clara e motivadora para o usuário

        2.  'dados_grafico_projecao': Um dicionário com os dados formatados para criar um gráfico de barras que represente a projeção da situação financeira do usuário após aplicar o método recomendado e seguir as mudanças sugeridas. **Este dicionário deve incluir todas as categorias de gasto presentes em 'dados_grafico_atual' (com os mesmos nomes), com seus valores ajustados de acordo com as orientações fornecidas. Novas categorias, como 'Poupança/Investimentos' ou 'Fundo de Emergência', podem ser adicionadas, desde que a soma de todos os valores projetados (incluindo as novas categorias) nunca ultrapasse o valor total da renda mensal.** Campos obrigatórios:
            'labels': Lista com os nomes das categorias de gasto projetadas (devem incluir todas as categorias de 'dados_grafico_atual' com os mesmos nomes, além de quaisquer novas categorias como 'Poupança/Investimentos').
            'valores': Lista com os valores em reais correspondentes a cada categoria, coerentes com os limites definidos pelo método e sem ultrapassar a renda. Os valores das categorias de gasto originais devem ser mantidos ou ajustados conforme as sugestões, mas as categorias devem permanecer.
            'titulo': Título do gráfico (ex: 'Projeção Financeira Após Mudanças').

        Certifique-se de que os dados nos dicionários `'dados_grafico_atual'` e `'dados_grafico_projecao'` estejam organizados de forma clara e lógica para facilitar a criação dos gráficos em um ambiente externo.
        """,
        description="Agente que orienta o usuário a como melhorar sua situação financeira.",
    )

    return call_agent(orientador, f"Dados: {dados}\nAnálise: {analise}")

In [ ]:
def valida_valor(valor):
  try:
    if type(valor) == str:
        valor = float(valor.replace(',', '.'))
    if valor <= 0:
      print('\n⚠️ Valor deve ser maior que zero.')
      return False
    return valor
  except ValueError:
    print('\n⚠️ Valor informado é invalido.')
    return False


In [ ]:
print('🚀 Iniciando o Sistema de Auxílio Financeiro com 4 Agentes 🚀')

renda_informada = False;
while not renda_informada:

  renda_mensal = valida_valor(input("\n❓ Por favor, digite a sua renda mensal (ex: 1500,00): "))
  if renda_mensal:
    print(f'\nRenda informada: R$ {renda_mensal}')
    renda_informada = True

gastos = {}
while True:
  nome_gasto = ''
  while not nome_gasto or nome_gasto.strip() == '' or nome_gasto.isdigit():
    nome_gasto = input("\n❓ Por favor, digite o nome do gasto que quer incluir (ex: moradia): ").strip()
    if not nome_gasto or nome_gasto.isdigit():
      print("⚠️ Nome inválido. Digite um nome de categoria válido (ex: moradia, transporte).")
  valor_gasto = 0
  while not valor_gasto:
    valor_gasto = valida_valor(input("❓ Por favor, digite o valor do gasto (ex: 600,00): "))
  if valor_gasto:
    gastos[nome_gasto] = valor_gasto
    print(f'\nGasto informado: {nome_gasto} - R$ {valor_gasto}')
    if len(gastos) >= 3:
      continuar = input("\n❓ Gostaria de incluir mais um tipo de gasto? (S/N) ")
      if continuar.upper() == 'S':
        continue
      else:
        break

print(f'\nRenda informada: R$ {renda_mensal}')
print(f'Gastos informados: {gastos}')
print('\nNossos agentes irão começar a analisar os dados...')

buscador = agente_buscador(renda_mensal)
print('\n✅ Melhores métodos para sua renda mensal encontrados!')

matematico = agente_matematico(renda_mensal, gastos)
print('\n✅ Cálculos realizados!')

analista = agente_analista(matematico, buscador)
print('\n✅ Análise realizada!')

orientador = agente_orientador(matematico, analista)

inicio_json = orientador.find('{')
fim_json = orientador.rfind('}')

# Extrair apenas a parte do JSON
if inicio_json != -1 and fim_json != -1:
    json_string = orientador[inicio_json : fim_json + 1]
    try:
        # Converter a string JSON para um dicionário Python
        resposta_orientador = json.loads(json_string)

        # Acessando a orientação para o usuário
        orientacao_usuario = resposta_orientador['orientacao']
        print('\n📈 Orientação do Agente Orientador 📈\n')
        display(Markdown(orientacao_usuario))

        # Acessando os dados para o gráfico da situação atual
        labels = list(gastos.keys())
        valores = list(gastos.values())

        # Gráfico da situação atual
        fig_atual = go.Figure()

        fig_atual.add_trace(go.Bar(
            x=labels,
            y=valores,
            name='Gastos Atuais',
            marker_color='indianred'
        ))

        fig_atual.update_layout(
            title="Situação Financeira Atual",
            xaxis_title="Categorias",
            yaxis_title="Valores (R$)",
            barmode='group'
        )

        fig_atual.show()

        print()

        dados_projecao = resposta_orientador['dados_grafico_projecao']
        labels_projecao = dados_projecao['labels']
        valores_projecao = dados_projecao['valores']
        titulo_projecao = dados_projecao['titulo']

        # Gráfico da projeção
        fig_proj = go.Figure()

        fig_proj.add_trace(go.Bar(
            x=dados_projecao['labels'],
            y=dados_projecao['valores'],
            name='Projeção',
            marker_color='royalblue'
        ))

        fig_proj.update_layout(
            title=dados_projecao['titulo'],
            xaxis_title="Categorias",
            yaxis_title="Valores (R$)",
            barmode='group'
        )

        fig_proj.show()

    except json.JSONDecodeError as e:
        print(f"Erro ao decodificar JSON: {e}")
        print(f"String JSON com problema: {json_string}")
else:
    print("Não foi possível encontrar um objeto JSON válido na string.")
